In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# TEC Historic Arbitrage Depth Analysis

> Using historic data to populate liquidity models and calculate arbitrage depth

In [ ]:
from conding.samm.uniswap import Uniswap
from conding.pamm.abc import AugmentedBondingCurve
from conding.dune.tec import TECDashboard
import pandas as pd

In [ ]:
tec = TECDashboard()

In [ ]:
honeyswap = tec.bonding_curve.total_liquidity_over_time_honeyswap_aggregated()[['block_day', 'day_liq', 'net_liq']]
honeyswap['block_day'] = pd.to_datetime(honeyswap['block_day'], utc=True)
honeyswap.head()

2023-05-13 09:17:41,144 INFO dune_client.base_client waiting for query execution 01H0AYEKBX8DVTTFJG1BZFWDA7 to complete: ExecutionState.EXECUTING
2023-05-13 09:17:46,979 INFO dune_client.base_client waiting for query execution 01H0AYEKBX8DVTTFJG1BZFWDA7 to complete: ExecutionState.EXECUTING


,block_day,day_liq,net_liq
0,2022-01-13 00:00:00+00:00,100.000000,100.000000
1,2022-01-15 00:00:00+00:00,-4.848705,95.151295
2,2022-01-21 00:00:00+00:00,-26.411040,68.740255
3,2022-01-24 00:00:00+00:00,-10.000000,58.740255
4,2022-01-25 00:00:00+00:00,5062.703964,5121.444219


In [ ]:
tec_mint_burn = tec.bonding_curve.TEC_total_supply()
tec_mint_burn['date'] = pd.to_datetime(tec_mint_burn['date'], utc=True)
tec_mint_burn.head()

,date,mint_burn,total_amount,total_supply
0,2022-01-13 00:00:00+00:00,MINT,1.700000e+03,1.700000e+03
1,2022-01-19 00:00:00+00:00,MINT,1.992737e+06,1.994437e+06
2,2022-01-25 00:00:00+00:00,MINT,4.371255e+05,2.431563e+06
3,2022-01-26 00:00:00+00:00,MINT,9.996543e+03,2.441559e+06
4,2022-01-27 00:00:00+00:00,MINT,7.169769e+03,2.448729e+06


In [ ]:
price_chart = tec.market_information.price_chart()
price_chart['block_date'] = pd.to_datetime(price_chart['block_date'], utc=True)
price_chart

,block_date,avg_mint_price,avg_burn_price,avg_mkt_price
0,2022-01-25 00:00:00+00:00,1.956345888613234,2.2216159017515933,2.3050899888800456
1,2022-01-26 00:00:00+00:00,2.893025644547676,2.2216159017515933,2.5920101807901506
2,2022-01-27 00:00:00+00:00,2.864333966225624,2.2216159017515933,2.5403510974065853
3,2022-01-28 00:00:00+00:00,2.8263603268647635,2.24996276540192,2.4776305971040147
4,2022-01-29 00:00:00+00:00,2.9437308337890533,2.2504243681326854,2.683241567655875
...,...,...,...,...
468,2023-05-09 00:00:00+00:00,0.5622850229364372,0.5622850229364372,0.5258280581358534
469,2023-05-10 00:00:00+00:00,0.5608069285035435,0.5608069285035435,0.5321566407096778
470,2023-05-11 00:00:00+00:00,0.5596423808136822,0.5596423808136822,0.5216465103908255
471,2023-05-12 00:00:00+00:00,0.5540953786399315,0.5540953786399315,0.4488854737838069


In [ ]:
liquidity = pd.merge(honeyswap, tec_mint_burn, left_on='block_day', right_on='date', how='outer')
liquidity.head()

,block_day,day_liq,net_liq,date,mint_burn,total_amount,total_supply
0,2022-01-13 00:00:00+00:00,100.000000,100.000000,2022-01-13 00:00:00+00:00,MINT,1700.000000,1.700000e+03
1,2022-01-15 00:00:00+00:00,-4.848705,95.151295,NaT,NaN,NaN,NaN
2,2022-01-21 00:00:00+00:00,-26.411040,68.740255,NaT,NaN,NaN,NaN
3,2022-01-24 00:00:00+00:00,-10.000000,58.740255,NaT,NaN,NaN,NaN
4,2022-01-25 00:00:00+00:00,5062.703964,5121.444219,2022-01-25 00:00:00+00:00,MINT,437125.512303,2.431563e+06


In [ ]:
liquidity = pd.merge(honeyswap, tec_mint_burn, left_on='block_day', right_on='date', how='outer')
liquidity['date'] = liquidity['date'].fillna(liquidity['block_day'])
# liquidity[['net_liq', 'total_supply']] = liquidity[['net_liq', 'total_supply']].ffill()
liquidity = liquidity.fillna(0)
liquidity = liquidity.drop('block_day',axis=1)
liquidity.sort_values('date')

,day_liq,net_liq,date,mint_burn,total_amount,total_supply
0,100.000000,100.000000,2022-01-13 00:00:00+00:00,MINT,1.700000e+03,1.700000e+03
1,-4.848705,95.151295,2022-01-15 00:00:00+00:00,0,0.000000e+00,0.000000e+00
1825,0.000000,0.000000,2022-01-19 00:00:00+00:00,MINT,1.992737e+06,1.994437e+06
2,-26.411040,68.740255,2022-01-21 00:00:00+00:00,0,0.000000e+00,0.000000e+00
3,-10.000000,58.740255,2022-01-24 00:00:00+00:00,0,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
1817,-22.573297,44648.161036,2023-05-12 00:00:00+00:00,BURN,-8.395300e+03,1.774614e+06
1821,-0.529340,44445.214466,2023-05-13 00:00:00+00:00,BURN,-1.114350e+02,1.774503e+06
1822,-2.349070,44442.865396,2023-05-13 00:00:00+00:00,BURN,-1.114350e+02,1.774503e+06
1823,0.529340,44443.394736,2023-05-13 00:00:00+00:00,BURN,-1.114350e+02,1.774503e+06


In [ ]:
state = pd.merge(liquidity, price_chart, left_on='date', right_on='block_date', how='outer').sort_values('date')
state[['avg_mint_price', 'avg_burn_price', 'avg_mkt_price']] = state[['avg_mint_price', 'avg_burn_price', 'avg_mkt_price']].ffill().bfill()
state = state.drop('block_date',axis=1).set_index('date')
state

,day_liq,net_liq,mint_burn,total_amount,total_supply,avg_mint_price,avg_burn_price,avg_mkt_price
date,,,,,,,,
2022-01-13 00:00:00+00:00,100.000000,100.000000,MINT,1.700000e+03,1.700000e+03,1.956345888613234,2.2216159017515933,2.3050899888800456
2022-01-15 00:00:00+00:00,-4.848705,95.151295,0,0.000000e+00,1.700000e+03,1.956345888613234,2.2216159017515933,2.3050899888800456
2022-01-19 00:00:00+00:00,0.000000,44445.743806,MINT,1.992737e+06,1.994437e+06,1.956345888613234,2.2216159017515933,2.3050899888800456
2022-01-21 00:00:00+00:00,-26.411040,68.740255,0,0.000000e+00,1.700000e+03,1.956345888613234,2.2216159017515933,2.3050899888800456
2022-01-24 00:00:00+00:00,-10.000000,58.740255,0,0.000000e+00,1.700000e+03,1.956345888613234,2.2216159017515933,2.3050899888800456
...,...,...,...,...,...,...,...,...
2023-05-12 00:00:00+00:00,-22.573297,44648.161036,BURN,-8.395300e+03,1.774614e+06,0.5540953786399315,0.5540953786399315,0.4488854737838069
2023-05-13 00:00:00+00:00,-0.529340,44445.214466,BURN,-1.114350e+02,1.774503e+06,0.5486815642541326,0.5486815642541326,0.5252311803074452
2023-05-13 00:00:00+00:00,-2.349070,44442.865396,BURN,-1.114350e+02,1.774503e+06,0.5486815642541326,0.5486815642541326,0.5252311803074452


In [ ]:
state.iloc[-5:]['total_supply'].astype(str)

date
2023-05-12 00:00:00+00:00    1774614.4012775936
2023-05-13 00:00:00+00:00    1774502.9662775935
2023-05-13 00:00:00+00:00    1774502.9662775935
2023-05-13 00:00:00+00:00    1774502.9662775935
2023-05-13 00:00:00+00:00    1774502.9662775935
Name: total_supply, dtype: object